In [3]:
import sys
sys.path.append('../')

import dataset_modules.preloaded_one_part_dataset as popd
import torch
import dataset_modules.image_preloader as image_preloader
import main
import importlib
import torchvision.transforms as transforms

importlib.reload(popd)

<module 'dataset_modules.preloaded_one_part_dataset' from '/Users/fcuevas/Documents/fing/tesis/jupyters/dataset_modules/../dataset_modules/preloaded_one_part_dataset.py'>

In [5]:
# preload images
preloaded_images = image_preloader.preload_images(
        images_folder='./imgs_final/', 
        state_file='./state.json', 
        resize_to=224
)

Started to preload images...
Image preloading complete in 2m 46s
The preloaded images use 0.140716552734375MB of memory


In [7]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv"
    )

----- CLASS DISTRIBUTION -----
Class: no_paragolpe_delantero, #1784, 55.39%
Class: paragolpe_delantero_roto, #1158, 35.95%
Class: paragolpe_delantero_sano, #279, 8.66%


In [8]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        state_file="./state.json",
        complaint_parts_file="../preprocessing/piezas_normalizadas.csv",
        use_selected_parts=True
    )

----- CLASS DISTRIBUTION -----
Class: no_paragolpe_delantero, #1784, 55.39%
Class: paragolpe_delantero_roto, #897, 27.85%
Class: paragolpe_delantero_sano, #540, 16.76%


In [33]:
train_dataset = popd.PreloadedOnePartDataset(
        "Paragolpe Delantero",
        preloaded_images,
        offline_augmentation=196,
        data_augmentation = transforms.Compose([
            transforms.RandomHorizontalFlip(p=1)
        ]),
        class_to_augment=0,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        remove_not_visible=True,
        ignore_repair=True,
        visibility_file='./dataset_modules/analisis_manual_paragolpe.csv',
        state_file="./dataset_modules/state-train.json",
    )

----- CLASS DISTRIBUTION -----
Class: paragolpe_delantero_sano, #416, 51.49%
Class: paragolpe_delantero_roto, #392, 48.51%
